In [ ]:
'''
What Is Association Rule Learning?
It identifies patterns like: “If a customer buys bread, they’re likely to buy butter.”

Common algorithms: Apriori, FP-Growth

Key metrics: Support, Confidence, Lift
'''

In [1]:
# Step 1: Install required libraries
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/4c/43/2fc7f76c8891aef148901f1ba3dee65c1cbac00a85ae5ee0dabc2b861256/mlxtend-0.23.4-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.4 MB 991.0 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.4 MB 2.5 MB/s eta 0:00:01
   ----------------- ---------------------- 0.6/1.4 MB 5.4 MB/s eta 0:00:01
   ----------------------- ---------------- 0.8/1.4 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------  1.4/1.4 MB 7.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 7.1 MB/s eta 0:00:00


In [2]:
# Step 2: Import libraries
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# Step 3: Load and preprocess data
#Dataset URL -- https://www.kaggle.com/code/shahraizanwar/association-rule-mining/input
data = pd.read_csv('D:\\Downloads\\Groceries_dataset\\Groceries_dataset.csv')
basket = data.groupby(['Member_number','Date'])['itemDescription'].apply(list).reset_index()
basket['itemDescription'] = basket['itemDescription'].apply(lambda x: list(set(x)))

In [4]:
# Step 4: One-hot encode the items
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_data = te.fit_transform(basket['itemDescription'])
df = pd.DataFrame(te_data, columns=te.columns_)

In [5]:
# Step 5: Apply Apriori algorithm
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

In [6]:
# Step 6: Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

Empty DataFrame
Columns: [antecedents, consequents, support, confidence, lift]
Index: []
